In [2]:
from __future__ import absolute_import, division, print_function

import pathlib
import pandas as pd
import seaborn as sns
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Dense, Dropout
from keras.models import Model

from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

import time

PYTHONHASHSEED=0
print(tf.__version__)

1.10.0


In [3]:
import scipy
import numpy
import matplotlib
import pandas
import sklearn
# import pydot
import h5py

import tensorflow
# import keras

print('scipy ' + scipy.__version__)
print('numpy ' + numpy.__version__)
print('matplotlib ' + matplotlib.__version__)
print('pandas ' + pandas.__version__)
print('sklearn ' + sklearn.__version__)
print('h5py ' + h5py.__version__)

print('tensorflow ' + tensorflow.__version__)
print('keras ' + keras.__version__)

scipy 1.1.0
numpy 1.13.3
matplotlib 1.5.3
pandas 0.23.0
sklearn 0.19.1
h5py 2.7.1
tensorflow 1.10.0
keras 2.1.6-tf


In [7]:
from keras import backend as K

K.tensorflow_backend._get_available_gpus()

[]

In [ ]:
train_dataset = pd.read_pickle('.pkl')
test_dataset = pd.read_pickle('.pkl')


In [ ]:
train_dataset.index = range(len(train_dataset.index))
test_dataset.index = range(len(test_dataset.index))


In [ ]:
# 결측치 변환 및 기초작업
# 결측치 중 nan 이면 0으로 대체
train_dataset.iloc[:, 0:20] = train_dataset.iloc[:, 0:20].fillna(0)
train_dataset.iloc[:, 53:73] = train_dataset.iloc[:, 53:73].fillna(0)

# 남은 변수의 경우 NA 경우 제거
train_dataset = train_dataset.dropna()

# 동일작업 test set에도 실시
test_dataset.iloc[:, 0:20] = test_dataset.iloc[:, 0:20].fillna(0)
test_dataset.iloc[:, 53:73] = test_dataset.iloc[:, 53:73].fillna(0)
test_dataset = test_dataset.dropna()

# train_dataset.shape # (203728, 35)
# test_dataset.shape  # (11518, 35)

# label 과 feature 분리
train_labels = train_dataset[train_dataset.columns[53:73]].copy()
test_labels = test_dataset[test_dataset.columns[53:73]].copy()

train_dataset = train_dataset.drop(train_dataset.columns[53:73], 1)
test_dataset = test_dataset.drop(test_dataset.columns[53:73], 1)

In [ ]:
# 1. train set
# 추후 1개월에 대해 카드 이력이 없는 사람을 제거
drop_y = train_labels.loc[train_labels.sum(axis=1) == 0, :].index
train_labels = train_labels.drop(drop_y)
train_dataset = train_dataset.drop(drop_y)

# 과거 3개월에 대해 카드 이력이 없는 사람 제거
x_labels = train_dataset[train_dataset.columns[0:20]].copy()
drop_x = x_labels.loc[x_labels.sum(axis=1) == 0, :].index
train_labels = train_labels.drop(drop_x)
train_dataset = train_dataset.drop(drop_x)

# 2. test set
# 추후 1개월에 대해 카드 이력이 없는 사람을 제거
drop_y = test_labels.loc[test_labels.sum(axis=1) == 0, :].index
test_labels = test_labels.drop(drop_y)
test_dataset = test_dataset.drop(drop_y)

# 과거 3개월에 대해 카드 이력이 없는 사람 제거
x_labels = test_dataset[test_dataset.columns[0:20]].copy()
drop_x = x_labels.loc[x_labels.sum(axis=1) == 0, :].index
test_labels = test_labels.drop(drop_x)
test_dataset = test_dataset.drop(drop_x)

In [ ]:
test_dataset.shape # (11518, 55)

In [ ]:
# 연령대 범주화

bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, np.inf]
names = ['0', '10', '20', '30', '40', '50', '60', '70', '70']

train_dataset['CLN_AGE_RANGE'] = pd.cut(train_dataset['CLN_AGE'], bins, labels=names)
test_dataset['CLN_AGE_RANGE'] = pd.cut(test_dataset['CLN_AGE'], bins, labels=names)

train_dataset['CLN_AGE_RANGE'] = train_dataset['CLN_AGE_RANGE'].astype(int)
test_dataset['CLN_AGE_RANGE'] = test_dataaset['CLN_AGE_RANGE'].astype(int)
train_dataset.dtypes
train_age = train_dataset.pop('CLN_AGE')
test_age = test_dataset.pop('CLN_AGE')

In [ ]:
# id 제거

train_name = train_dataset.pop('key')
test_name = test_dataset.pop('key')
train_datset = train_dataset.drop(['CLNN'], axis=1)
test_datset = test_dataset.drop(['CLNN'], axis=1)

# 날짜 int로 변경
train_dataset['date'] = train_dataset['date'].astype(str).str.slice(4, 6)
test_dataset['date'] = test_dataset['date'].astype(str).str.slice(4, 6)
train_dataset['date'] = train_dataset['date'].astype(int)
test_dataset['date'] = test_dataset['date'].astype(int)

# 범주변수 one hot encoding하는 함수

def make_dummy(df):
    colnames = []
    for col in df.columns:
        if ( df[col].dtype == object):
            colnames.append(col)]
    dataset = pd.get_dummies(df, columns=colnames, prefix=colnames)    
    return dataset

train_dataset = make_dummy(train_dataset)
test_dataset = make_dummy(test_dataset)

test_dataset = test_dataset.drop(test_dataset.columns[26], 1)

In [ ]:
# 이상치 제거
# 'SAA_y'에 음수가 있어 0으로 변환
train_dataset['SAA_y'] = train_dataset['SAA_y'].clip(lower=0)
test_dataset['SAA_y'] = test_dataset['SAA_y'].clip(lower=0)

In [ ]:
# 

In [ ]:
diff_col = test_dataset.columns.difference(train_dataset.columns)
test_dataset = test_dataset.drop(diff_col, 1)

diff_col = train_dataset.columns.difference(test_dataset.columns)
train_dataset = train_dataset.drop(diff_col, 1)

In [ ]:
train_dataset.shape # (135517, 66)

In [ ]:
test_dataset.shape # (11518, 66)

In [ ]:
train_stats = train_dataset.describe()
train_stats = pd.DataFrame(train_stats.transpose())
train_stats

In [ ]:
for i in range(len(train_stats.index)):
    print(i, train_stats.index[i] )

In [ ]:
# normalization for continuous variable

for i in range(0, 40):
    column = train_stats.index[1]
    train_dataset.loc[:, column] = ( train_dataset.loc[:, column] - train_stats['mean'][i] ) / train_stats['std'][i]
    test_dataset.loc[:, column] = ( test_dataset.loc[:, column] - test_stats['mean'][i] ) / test_stats['std'][i]
    
for i in [46, 47, 48]:
    column = train_stats.index[i]
    train_dataset.loc[:, column] = ( train_dataset.loc[:, column] - train_stats['mean'][i] ) / train_stats['std'][i]
    test_dataset.loc[:, column] = ( test_dataset.loc[:, column] - test_stats['mean'][i] ) / test_stats['std'][i]


In [ ]:
train_scaled = train_dataset
test_scaled = test_dataset
input_dataset = pd.DataFrame(train_scaled)

In [ ]:
def build_model(h1, h2, h3, 1r):
    model = keras.Sequential([
        layers.Dense(h1, activation = tf.nn.relu, input_shape=[len(pd.DataFrame(input_dataset).keys())], 
                    bias_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=.05, seed=123) ),
        layers.Dense(h2, activation = tf.nn.relu, bias_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=.05, seed=123)),
        layers.Dense(h3, activation = tf.nn.relu, bias_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=.05, seed=123)),
        layers.Dense(20, activation = 'sigmoid')
    ])
    
    optimizer = keras.optimizers.RMSprop(r)
    
    model.compile(
        loss ='binary_crossentropy',
        optimizer = optimizer,
        metrics = ['accuracy']
    )
    
    return model

class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, )